<a href="https://colab.research.google.com/github/salta-ak/Google-trend-forecast-with-LSTM/blob/main/multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install keras-tuner -q
!pip3 install pytrends

     |████████████████████████████████| 96 kB 2.9 MB/s 


In [ ]:
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers
from keras_tuner import RandomSearch
from keras_tuner import BayesianOptimization
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from keras.layers import SimpleRNN, GRU
from keras.callbacks import CSVLogger
import sys
import csv
import os
import pandas as pd
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360)
import matplotlib.pyplot as plt
# Load data from with pytrend API



In [ ]:
filename=["3_kword",'bitcoin','covid','stock_price']
kw_list = ["bitcoin","COVID-19","stock price"]
btc_list=["bitcoin","cryptocurrency","blockchain+bitcoin"]
st_list=["stock+price","stock+index","futures+stock"]
cov_list=["Covid","cough+covid","symptoms+covid"]
kw_all=[kw_list,btc_list,st_list,cov_list]
for i,j in zip(kw_all,filename):
    pytrends.build_payload(i, cat=0, timeframe='today 5-y', geo='', gprop='')
    data=pytrends.interest_over_time()
    data=data.loc[~(data==0.0).all(axis=1)]
    series = data.iloc[: , :-1].astype(float).sort_index()
    series.to_csv('{}.csv'.format(j))

In [ ]:
def to_supervised(data,dropNa = True,lag = 1):
    df = pd.DataFrame(data)
    column = []
    column.append(df)
    for i in range(1,lag+1):
        column.append(df.shift(-i))
    df = pd.concat(column,axis=1)
    df.dropna(inplace = True)
    features = data.shape[1]
    df = df.values
    supervised_data = df[:,:features*lag]
    supervised_data = np.column_stack( [supervised_data, df[:,features*lag]])
    return supervised_data

In [ ]:
#sys.stdout = open("output.txt", "w")
look_back=(12,24)



  #building LSTM model 
for i in filename:
  df = pd.read_csv(str(i)+'.csv', sep=',')
  df['date'] = pd.to_datetime(df['date'])
  df.set_index('date', inplace=True)
  appended_df = []
  
  for window_size in look_back:
    for neuron in (4,8,16,32,60):
      values = df.values
      scaler = MinMaxScaler()
      scaled = scaler.fit_transform(values)
      supervised = to_supervised(scaled,lag=window_size)
      features = df.shape[1]
      train_len = df.shape[0]*0.8
      train_len = int(df.shape[0]*0.8)
      X = supervised[:,:supervised.shape[1]-1]
      y = supervised[:,supervised.shape[1]-1:]

      x_train = X[:train_len,:]
      x_test = X[train_len:,:]
      y_train = y[:train_len]
      y_test = y[train_len:]
      x_train = x_train.reshape(x_train.shape[0], 1, window_size*features)
      x_test = x_test.reshape(x_test.shape[0], 1, window_size*features) 
      

      model = Sequential()
      model.add(SimpleRNN(neuron, input_shape=(1, window_size*features)))
      model.add(Dense(1))
      print(model.summary())


      model.compile(loss='mean_squared_error', optimizer='adam')
      path_checkpoint = "model_checkpoint"+str(i)+str(window_size)+str(neuron)+".5h"
      es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)
      modelckpt_callback = keras.callbacks.ModelCheckpoint( monitor="val_loss",filepath=path_checkpoint,  verbose=0,  save_weights_only=True, save_best_only=True)
      es_callback = CSVLogger(str(i)+"_"+str(window_size)+"_"+str(neuron)+'log.csv', append=True, separator=';')
  
      history =  model.fit( x_train,y_train, validation_data = (x_test,y_test), epochs = 100 , batch_size = 1, verbose = 2, callbacks=[es_callback, modelckpt_callback,])

      #scale back the prediction to orginal scale
      y_pred = model.predict(x_test)
      x_test = x_test.reshape(x_test.shape[0],x_test.shape[2]*x_test.shape[1])

      inv_new = np.concatenate( (y_pred, x_test[:,-2:] ) , axis =1)
      inv_new = scaler.inverse_transform(inv_new)
      final_pred = inv_new[:,0]

      y_test = y_test.reshape( len(y_test), 1)
      inv_new = np.concatenate( (y_test, x_test[:,-2:] ) ,axis = 1)
      inv_new = scaler.inverse_transform(inv_new)
      actual_pred = inv_new[:,0]    
      error = np.sqrt(mean_squared_error(final_pred, actual_pred))
      print('Test RMSE: %.3f' % error)    
      print('kword: '+ str(i))      
      print('window size: '+ str(window_size))
      print('neuron: '+ str(neuron))
    
sys.stdout.close()  

In [ ]:
!zip -r /content/file.zip /content/*

updating: content/3_kword_12_16log.csv (deflated 53%)
updating: content/3_kword_12_32log.csv (deflated 53%)
updating: content/3_kword_12_4log.csv (deflated 53%)
updating: content/3_kword_12_60log.csv (deflated 53%)
updating: content/3_kword_12_8log.csv (deflated 53%)
updating: content/3_kword_24_16log.csv (deflated 53%)
updating: content/3_kword_24_32log.csv (deflated 53%)
updating: content/3_kword_24_4log.csv (deflated 53%)
updating: content/3_kword_24_60log.csv (deflated 53%)
updating: content/3_kword_24_8log.csv (deflated 53%)
updating: content/3_kword.csv (deflated 78%)
updating: content/bitcoin_12_16log.csv (deflated 53%)
updating: content/bitcoin_12_32log.csv (deflated 53%)
updating: content/bitcoin_12_4log.csv (deflated 54%)
updating: content/bitcoin_12_60log.csv (deflated 53%)
updating: content/bitcoin_12_8log.csv (deflated 53%)
updating: content/bitcoin_24_16log.csv (deflated 53%)
updating: content/bitcoin_24_32log.csv (deflated 53%)
updating: content/bitcoin_24_4log.csv (defl

In [ ]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>